For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) &+ h'(F)(C(F)-P(F)) \\
      &+ \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}

  
To implement Breeden-Litzenberger formula (or static replication in general) in Python, we need to be able to handle numerical integration for the put/call integrals.

We need to use numerical integration to evaluate the formula. This can be done in Python easily. For example, consider the integral

\begin{equation*}
    \begin{split}
      \int_0^1 x^2 \; dx = \frac{1}{3}
    \end{split}
\end{equation*}

We can evaluate it in Python as follows:

In [5]:
from scipy.integrate import quad


def integrand(x):
    return x**2


I = quad(integrand, 0.0, 1.0)
print('The integral is: %.9f' % I[0])


The integral is: 0.333333333


We can test our static replication implementation with

\begin{equation*}
\begin{split}
    \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}

Suppose we assume that market is following Black76 model, then we have

In [14]:
import numpy as np
from scipy.stats import norm


def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)


def callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price


def putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price


S = 100.0
r = 0.05
T = 1.0
sigma = 0.4
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)


The expected integrated variance is: 0.160000000


Since we are using Black76 model in the above, we can cross-check our implementation with the integrated variance based on Black76 model, since under Black76 (or Black-Scholes) model, volatility is deterministic.
  
Hence we have
\begin{equation*}
  \begin{split}
    \mathbb{E}\left[ \int_0^T \sigma_t^2 \; dt \right] &= \int_0^T \sigma^2 \; dt \\
    &= \sigma^2T
  \end{split}
\end{equation*}

So for instance if $\sigma=0.4$ and $T=1$, then the integrated variance should be $\sigma^2T = 0.16$.

Once tested, we should replace BlackScholes with SABR model in order to accurately capture the implied volatility smiile/skew in the option market.